# Systems for Processing Big Data
## TP2 - Energy Meter Live Monitoring


The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

For all the sensors combined:

1. For the current month and current day, compute the running total energy consumed so far. The values should be updated every 5 minutes.

2. For the current month and current day, compute the running total energy consumed so far, **as a percentage**, **compared to the same periods in February 2024**. The values should be updated every 5 minutes.

For each sensor, separately:

3. For the current month and current day, compute the running total energy consumed so far, as a percentage, **comparing the value of each individual sensor, relative to the same results for all the sensors together (as in #1)**. The values should be updated every 5 minutes. (Sorted in descending order by value and sensor.)



## Requeriments

Solve each question using Structured Spark Streaming.

---
### Colab Setup


In [ ]:
#@title Install PySpark
!pip install pyspark --quiet

In [ ]:
#@title Download Archived February Energy Readings
!wget -q -O /tmp/readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!grep "2024-02" /tmp/readings.csv > february-energy-readings.csv
!head -2 february-energy-readings.csv


2024-02-01 00:00:00;D;2615.0
2024-02-01 00:00:18;C;1098.8


In [ ]:
#@title Start the Structured Source
!wget -q -O - https://github.com/smduarte/spbd-2425/raw/main/scripts/json_energy_sender.tgz  | tar xfz - 2> /dev/null

!nohup python json_energy_sender/server.py --filename json_energy_sender/energy-readings.csv --speedup 60 > /dev/null 2> /dev/null &


Note: --speedup 60, means the stream is played 60x faster than realtime. Therefore, 1 second in real time corresponds to 1 minute of stream data.


# Question 1

## For all the sensors combined: For the current month and current day, compute the running total energy consumed so far. The values should be updated every 5 minutes.

First, we use window function to group the data by daily time windows (1-day window) and sensor.
For each day we take the maximum and minimum energy readings for each sensor and we calculate their running energy by taking the difference of max and min.

---

Then, we further procces each batch in the following way:



*   **Daily running total calculation**

We calculate the daily running totals as a sum of all the energy usage across all sensors.

*   **Monthly running total calculation**

We use Window partitioning, to divide the data by month, to ensure calculations for each month separately, and then we order it by day.

Then, we calculate running totals for each month up to the current day - we take the sum of energy usage from the beginning of the partition (the beginning of the month), until the current row (current day)

---
Finally, we select the appropriate columns, such as date, daily running total and monthly running total

After collecting and processing incoming data every 5 seconds for 15minutes, we finally got such result table:

```
+----------+-------------------+---------------------+
|      date|daily_running_total|monthly_running_total|
+----------+-------------------+---------------------+
|2024-10-01|              41.68|                41.68|
|2024-10-02|              26.02|                 67.7|
|2024-10-07|               0.37|                68.07|
|2024-10-08|              52.42|               120.49|
|2024-10-09|              55.09|               175.58|
|2024-10-10|              43.53|               219.11|
|2024-10-11|              44.34|               263.45|
|2024-10-12|              53.17|               316.62|
|2024-10-13|              65.27|               381.89|
|2024-10-14|              51.79|               433.68|
|2024-10-15|               51.9|               485.58|
|2024-10-16|              48.25|               533.83|
|2024-10-17|              55.89|               589.72|
|2024-10-18|               57.5|               647.22|
|2024-10-19|              11.71|               658.93|
|2024-10-25|              25.63|               684.56|
|2024-10-26|              53.13|               737.69|
|2024-10-27|              69.78|               807.47|
|2024-10-28|              62.82|               870.29|
|2024-10-29|              56.43|               926.72|
|2024-10-30|              64.57|               991.29|
|2024-10-31|              68.53|              1059.82|
|2024-11-01|              80.64|                80.64|
|2024-11-02|              59.67|               140.31|
|2024-11-03|              69.72|               210.03|
|2024-11-04|              64.16|               274.19|
|2024-11-05|              62.08|               336.27|
+----------+-------------------+---------------------+
```


In [ ]:
def get_running_totals(df):

    """
    Function to compute daily and monthly running totals of energy consumption.

    Input:
    - df: DataFrame with the following columns:
        - "window.start": Start timestamp of the aggregation window
        - "window.end": End timestamp of the aggregation window
        - "running_total": Aggregated energy value for each sensor

    Output:
    - Returns a DataFrame with the following columns:
        - "date": The date corresponding to each day in the data
        - "daily_running_total": Total energy consumed for each day (summed across all sensors)
        - "monthly_running_total": Total energy consumed within each month from the first day up to the current day
    """

    # Calculate running total for each day - summed over all sensors
    df = df.groupBy("window.start", "window.end")\
          .agg(round(sum("running_total"), 2).alias("daily_running_total"))\
          .withColumn("day", day("start"))\
          .withColumn("month", month("start"))

    # Partition by month, to ensure calculation for each month separately, and order it by day
    window_spec = Window.partitionBy("month").orderBy("day").rowsBetween(Window.unboundedPreceding, Window.currentRow)

    # Calculate running total for each month - the sum of energy usage from the beginning of the partition (the beginning of the month), until the current row (current day)
    df_totals = df.withColumn("monthly_running_total", round(sum("daily_running_total").over(window_spec), 2))

    #Select appropriate columns
    df_totals = df_totals.withColumn("date", to_date("start")) \
                                  .select("date", "daily_running_total", "monthly_running_total") \
                                  .orderBy("date")

    return df_totals

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample") \
    .getOrCreate()


# Extract a sample JSON string to infer schema
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)


# Create DataFrame representing the stream of input
# lines from connection to logsender 7777
try:
  json_lines = spark.readStream.format("socket") \
      .option("host", "localhost") \
      .option("port", 7777) \
      .load()

   # Parse JSON and extract date components
  parsed_df = json_lines \
      .withColumn("json_data", from_json(col("value"), inferred_schema)) \
      .select("json_data.*") \
      .withColumn("date", to_timestamp("date")) \

  # Get running totals for each day and each sensor
  running_totals = parsed_df \
                            .groupBy(window("date", "1 day"), "sensor",) \
                            .agg(
                                  max("energy").alias("max_energy"),
                                  min("energy").alias("min_energy")
                            ) \
                            .withColumn("running_total", round(col("max_energy") - col("min_energy"), 4)
                            )

  # Output the results with 5-minute trigger
  query = running_totals \
      .writeStream \
      .outputMode("complete") \
      .trigger(processingTime='5 minutes') \
      .foreachBatch(lambda df, epoch: get_running_totals(df).show(25))\
      .start()

  query.awaitTermination(600)
except Exception as err:
  print(err)
  query.stop()

+----+-------------------+---------------------+
|date|daily_running_total|monthly_running_total|
+----+-------------------+---------------------+
+----+-------------------+---------------------+

+----------+-------------------+---------------------+
|      date|daily_running_total|monthly_running_total|
+----------+-------------------+---------------------+
|2024-10-01|               4.07|                 4.07|
+----------+-------------------+---------------------+

+----------+-------------------+---------------------+
|      date|daily_running_total|monthly_running_total|
+----------+-------------------+---------------------+
|2024-10-01|              11.88|                11.88|
+----------+-------------------+---------------------+

+----------+-------------------+---------------------+
|      date|daily_running_total|monthly_running_total|
+----------+-------------------+---------------------+
|2024-10-01|              19.33|                19.33|
+----------+-------------------

# Question 2

## For all the sensors combined: For the current month and current day, compute the running total energy consumed so far, **as a percentage, compared to the same periods in February 2024.** The values should be updated every 5 minutes.

## Approach
The first step for this task was to prepare the february readings such that they can be used later for computing the percentage of the current running total of the same period in february.

in particular, for all sensors combined, the following data was computed:

```JSON
+---+----------+-----------------+-------------------------+-----------------+
|day|  feb_date|feb_running_total|feb_monthly_running_total|max_monthly_total|
+---+----------+-----------------+-------------------------+-----------------+
|  1|2024-02-01|            119.7|                    119.7|           1971.5|
|  2|2024-02-02|             69.6|                    189.3|           1971.5|
|  9|2024-02-09|             55.7|                    245.0|           1971.5|
| 10|2024-02-10|            113.9|                    358.9|           1971.5|
| 11|2024-02-11|            117.7|                    476.6|           1971.5|
+---+----------+-----------------+-------------------------+-----------------+
```

**Day** Represents the day of the month this is later used to join with the streamd data table.

**feb_running_total** is the daily running total for all sensors combined

**feb_monthly_running_total** is the accumulated running total for the whole month. It is a growing number that contains sum of the daily running totals first day of the month until the current day in the data. So for the 2024-02-02 the feb_monthly_running_total is the sum of 119.7 + 69.6 = 189.3.

**max_monthly_total** is the maximum of the feb_monthly_running_total. This represents the the total energy consumned in the month of february.

In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    # Load the februaray readings and prepare them for each sensor and day
    readings = spark.read.csv('february-energy-readings.csv', sep =';', header=False)\
                                .withColumnRenamed("_c0", "date")\
                                .withColumnRenamed("_c1", "sensor")\
                                .withColumnRenamed("_c2", "energy")

    # filter the readings to only contain data from february
    readings_february = readings.filter("date >= date '2024-02-01' AND date < date '2024-03-01'")

    # compute the february running totals for all sensors combined
    february_running_total = readings_february.withColumn("feb_date", to_date("date"))\
        .groupBy("sensor", "feb_date")\
        .agg(
            max("energy").alias("current_total_energy"),
            min("energy").alias("initial_total_energy")
        )\
        .withColumn("feb_running_total",
                    round(col("current_total_energy") - col("initial_total_energy"), 4)
        )\
        .groupBy("feb_date") \
        .agg(round(sum("feb_running_total"), 2).alias("feb_running_total"))\
        .withColumn('day', day("feb_date"))\
        .withColumn('month', month("feb_date"))\
        .orderBy("day")\
        .select("feb_date", "day", "month", "feb_running_total")

    # Partition by month, to ensure calculation for each month separately, and order it by day
    window_spec = Window.partitionBy("month").orderBy("day").rowsBetween(Window.unboundedPreceding, Window.currentRow)

    # Calculate running total for each month - the sum of energy usage from the beginning of the partition (the beginning of the month), until the current row (current day)
    df_totals = february_running_total.withColumn("feb_monthly_running_total", round(sum("feb_running_total").over(window_spec), 2))

    #Select appropriate columns
    df_february_running_totals = df_totals.withColumn("start_date", to_date("feb_date")) \
                                  .select("day", "feb_date", "feb_running_total", "feb_monthly_running_total") \
                                  .orderBy("feb_date")

    # Add a column that represents the maximum energy which corresponds to the last day of the month
    df_february_running_totals = df_february_running_totals.withColumn(
        "max_monthly_total",
        max("feb_monthly_running_total").over(Window.partitionBy(lit(1)))
    )

    df_february_running_totals.show(5)

except Exception as err:
    print(err)

+---+----------+-----------------+-------------------------+-----------------+
|day|  feb_date|feb_running_total|feb_monthly_running_total|max_monthly_total|
+---+----------+-----------------+-------------------------+-----------------+
|  1|2024-02-01|            119.7|                    119.7|           1971.5|
|  2|2024-02-02|             69.6|                    189.3|           1971.5|
|  9|2024-02-09|             55.7|                    245.0|           1971.5|
| 10|2024-02-10|            113.9|                    358.9|           1971.5|
| 11|2024-02-11|            117.7|                    476.6|           1971.5|
+---+----------+-----------------+-------------------------+-----------------+
only showing top 5 rows



## Q2 Approach

Similar to Q1, the streamed data was aggreated using a widow with the width of 1 day.

For every sensor separtely, the daily running total is computed as the difference between the minimum and maximum reading for a particular day.

Before joining the output table with the february readings, the output table data was transformed to match with the shape of the february readings table.
This means, besides the running total for all sensors combined, also the monthly running total was computed. See prepare_percentage_of_february_readings

The transformed output table is then joined with the previously perpared february readings. The dataframes were joind by the day of the month. Before printing the table, it was filtered to remove columns that contain NULL values. This happened because not for every day of february and or october, data was available. These missing values were not considered for computing resutls.

The final output table looks like this:

```
+----------+---------------------------+-----------------------------+-------------------------------------+
|      date|daily_comparison_percentage|monthly_comparison_percentage|portion_of_total_february_consumption|
+----------+---------------------------+-----------------------------+-------------------------------------+
|2024-10-01|                      34.82|                        34.82|                                 2.11|
|2024-10-02|                      37.39|                        35.76|                                 3.43|
|2024-10-09|                       98.9|                        71.67|                                 8.91|
|2024-10-10|                      38.22|                        61.05|                                11.11|
|2024-10-11|                      37.67|                        55.28|                                13.36|
|2024-10-12|                      48.12|                        53.93|                                16.06|
|2024-10-13|                      58.02|                        54.59|                                19.37|
|2024-10-14|                      55.75|                        54.72|                                 22.0|
|2024-10-15|                       5.27|                        50.14|                                22.21|
+----------+---------------------------+-----------------------------+-------------------------------------+

```
**daily_comparison_percentage** contains the fraction of the current daily running energy compared to the same day in february. The value is a positive integer that indicats the percentage. NOTE: it is possible that the value beocomes larger than 100. In this case, one can see, that on this day, more than 100% of the energy was used than on the same day in february.

**monthly_running_total** represents the percentage of the current monthly running total compared to the same period of february. This value can be understood as. From the first day of the month until the current day, x% of the enegry was used than in the same period in february. The value is not steadyly increasing but can vary depending on the daily energy readings.

**portion_of_total_february_consumption** represents the fraction of energy of the current monthly runnint total compared to the total consumption of the month february. This value can be interperted as follows. From the first day until of the current month unitl now, x% of the energy was consumed as in the whole month of februaray. This number is steadily increasing.

In [ ]:
def prepare_percentage_of_february_readings(df: DataFrame, df_february: DataFrame):
    """
    Function to compute daily and monthly percentage energy consumption for each sensor.
    Takes two dataframes, 1st is the output table of the data sream processing and the 2nd is the reference month in this case february.

    Uses the reference month to compute the percentage of energy consumend in the current month compared to the reference month.
    This is done for the daiyl running total as well as for the monthly running total and the overall percentage of the total energy
    consumned compared to the total energy consumed in the reference month.

    @param: df: DataFrame with columns:
        - "window": The aggregation window start and end times
        - "sensor": Id for each sensor
        - "running_total": daily running total for each sensor

    @param df_february: DataFrame with columns:
        - "day": Day of the month
        - "feb_date": Start timestamp of the aggregation window
        - "feb_running_total": Sum across all daily running totals for all sensors
        - "feb_month_running_total": Sum across feb_running_total from first day of february until current feb_date
        - "max_monthly_total": global maximum runnint_total (the same value for all entries)
    """

    # Calculate the aggregated running total across all sensors and days in the range of the window
    df = df.groupBy("window.start", "window.end")\
          .agg(round(sum("running_total"), 2).alias("daily_running_total"))\
          .withColumn("day", day("start"))\
          .withColumn("month", month("start"))

    # Partition by month, to ensure calculation for each month separately, and order it by day
    window_spec = Window.partitionBy("month").orderBy("day").rowsBetween(Window.unboundedPreceding, Window.currentRow)

    # Calculate running total for each month - the sum of energy usage from the beginning of the partition (the beginning of the month), until the current row (current day)
    df_totals = df.withColumn("monthly_running_total", round(sum("daily_running_total").over(window_spec), 2))

    return_df = df_totals.join(df_february, on=['day'], how='left')\
        .withColumn("daily_comparison_percentage", round(col("daily_running_total") / col("feb_running_total") * 100, 2))\
        .withColumn("monthly_comparison_percentage", round(col("monthly_running_total") / col("feb_monthly_running_total") * 100, 2))\
        .withColumn("portion_of_total_february_consumption", round(col("monthly_running_total") / col("max_monthly_total") * 100, 2))\
        .withColumn("date", to_date("start"))\
        .na.drop(subset=["daily_comparison_percentage"])\
        .select("date", "daily_comparison_percentage", "monthly_comparison_percentage", "portion_of_total_february_consumption")\
        .orderBy("date")

    return return_df

In [ ]:
from datetime import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window


spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample2") \
    .config("spark.sql.streaming.statefulOperator.checkCorrectness.enabled", "false") \
    .getOrCreate()

# Extract a sample JSON string to infer schema
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)

try:
    json_lines = spark.readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 7777) \
        .load()

    # Parse JSON and extract date components
    parsed_df = json_lines \
        .withColumn("json_data", from_json(col("value"), inferred_schema)) \
        .select("json_data.*") \
        .withColumn("date", to_timestamp("date")) \

    running_totals = parsed_df.groupBy(
        window("date", "1 day"),
        "sensor",
    ).agg(
        max("energy").alias("max_energy"),
        min("energy").alias("min_energy")
    ).withColumn(
        "running_total",
        round(col("max_energy") - col("min_energy"), 4)
    )

    # Output the results with 5-minute trigger
    query = running_totals \
        .writeStream \
        .outputMode("complete") \
        .trigger(processingTime='5 minutes') \
        .foreachBatch(
            lambda df, epoch: prepare_percentage_of_february_readings(df, df_february_running_totals).show()
          )\
        .start()

    query.awaitTermination(600)
except Exception as err:
    print(err)
    query.stop()

+----+---------------------------+-----------------------------+-------------------------------------+
|date|daily_comparison_percentage|monthly_comparison_percentage|portion_of_total_february_consumption|
+----+---------------------------+-----------------------------+-------------------------------------+
+----+---------------------------+-----------------------------+-------------------------------------+

+----------+---------------------------+-----------------------------+-------------------------------------+
|      date|daily_comparison_percentage|monthly_comparison_percentage|portion_of_total_february_consumption|
+----------+---------------------------+-----------------------------+-------------------------------------+
|2024-10-01|                      34.82|                        34.82|                                 2.11|
|2024-10-02|                      37.39|                        35.76|                                 3.43|
+----------+---------------------------+--

# Question 3

## For each sensor, separately: For the current month and current day, compute the running total energy consumed so far, as a percentage, **comparing the value of each individual sensor, relative to the same results for all the sensors together (as in #1).** The values should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

We started similarly as in task #1 - we grouped the data by daily time windows and sensor and for each sensor we got their daily running energies.

As defined in function *get_global_running_totals*, we summed these values across all sensors to calculate the global daily running total and we partitioned the data by month to compute the running energy consumption up to the current day, but within one month.

**Percentage Calculations:**

As defined in function *get_percentages* :

*   For each sensor, we calculated its daily percentage contribution as its daily running total divided by the global daily total
*   Similarly, we calculated the monthly percentage contribution as its running monthly total divided by the global monthly total. To be able to that, firstly we caluclated running monthly total for each sensor, up to the current day, but within the same month (defined in function *get_sensors_monthly_totals*)

At the end, we joined all the neccesary tables, to get the desirable results, selecting the appropriate columns.


In [ ]:
def get_global_running_totals(df):
  """
    Function to compute daily and monthly running totals of energy consumption.

    Input:
    - df: DataFrame with columns:
        - "window.start": Start timestamp of the aggregation window
        - "window.end": End timestamp of the aggregation window
        - "sensor_running_total": Aggregated energy value for each sensor

    Output:
    - Returns a DataFrame with columns:
        - "date": The date corresponding to each day in the data
        - "daily_running_total": Total energy consumed for each day (summed across all sensors)
        - "monthly_running_total": Total energy consumed within each month from the first day up to the current day
    """

  # Calculate running total for each day - summed over all sensors
    df_totals = df.groupBy("window.start", "window.end")\
          .agg(round(sum("sensor_running_total"), 2).alias("daily_running_total"))\
          .withColumn("day", day("start"))\
          .withColumn("month", month("start"))

    # Partition by month, to ensure calculation for each month separately, and order it by day
    window_spec = Window.partitionBy("month").orderBy("day").rowsBetween(Window.unboundedPreceding, Window.currentRow)

    # Calculate running total for each month - the sum of energy usage from the beginning of the partition (the beginning of the month), until the current row (current day)
    df_totals = df_totals.withColumn("monthly_running_total", round(sum("daily_running_total").over(window_spec), 2)) \
                        .withColumn("date", to_date("start"))  \
                        .select("date", "month", "daily_running_total", "monthly_running_total")

    return df_totals


In [ ]:
def get_sensors_monthly_totals(df):
  """
    Function to compute monthly running totals of energy consumption for each sensor.

    Input:
    - df: DataFrame with columns:
        - "window.start": Start timestamp of the aggregation window
        - "sensor": Id for each sensor
        - "sensor_running_total": Aggregated daily energy value for each sensor

    Output:
    - Returns a DataFrame with columns:
        - "sensor_s": Sensor id
        - "date_s": Date corresponding to each data point
        - "sensor_monthly_running_total": Total energy consumed by each sensor from the start of the month up to the current day
    """

  # Partition by month and sensor, to ensure calculations for each sensor in each month separately, and order it by day
  window_spec = Window.partitionBy("month_s", "sensor").orderBy("day").rowsBetween(Window.unboundedPreceding, Window.currentRow)

  # Compute monthly running total for each sensor
  df_sensors_monthly = df.withColumn("month_s", month("window.start")) \
        .withColumn("day", day("window.start")) \
        .withColumn("sensor_monthly_running_total", round(sum("sensor_running_total").over(window_spec), 2)) \
        .withColumn("date_s", to_date("window.start")) \
        .withColumnRenamed("sensor", "sensor_s") \
        .select("sensor_s", "date_s", "sensor_monthly_running_total")

  return df_sensors_monthly


In [ ]:
def get_percentages(df):
  """
    Function to compute daily and monthly percentage energy consumption for each sensor.

    Input:
    - df: DataFrame with columns:
        - "window.start": Start timestamp of the aggregation window
        - "sensor": Id for each sensor
        - "sensor_running_total": Aggregated energy value for each sensor

    Output:
    - Returns a DataFrame with columns:
        - "date": Date corresponding to each data point
        - "sensor": Sensor id
        - "percentage_daily": Sensor's percentage contribution to daily energy consumption
        - "percentage_monthly": Sensor's percentage contribution to monthly energy consumption
        - "sensor_running_total": Daily energy consumption for a sensor
        - "sensor_monthly_running_total": Monthly energy consumption for a sensor (from the beginning of the month to the current date)
    """

    # Get daily and monthly global running totals
    df_totals = get_global_running_totals(df)

    # Get monthly running total for each sensor
    df_sensors_monthly = get_sensors_monthly_totals(df)

    # Compute daily percentage usage for each sensor
    df_percentages = df_totals.join(df, df["date_s"] == df_totals["date"], how="left") \
                        .withColumn("percentage_daily", round(col("sensor_running_total") / col("daily_running_total") * 100, 2)) \
                        .select("date", "month", "sensor", "sensor_running_total", "daily_running_total", "monthly_running_total", "percentage_daily")

    # Compute monthly percentage usage for each sensor
    df_percentages = df_sensors_monthly.join(df_percentages, (df_sensors_monthly["sensor_s"] == df_percentages["sensor"]) & (df_sensors_monthly["date_s"] == df_percentages["date"]), how="left") \
                        .withColumn("percentage_monthly", round(col("sensor_monthly_running_total") / col("monthly_running_total") * 100, 2)) \
                        .select("date", "sensor", "percentage_daily", "percentage_monthly", "sensor_running_total", "sensor_monthly_running_total")\
                        .orderBy("date", desc("percentage_daily"), desc("percentage_monthly"), "sensor")

    return df_percentages


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample") \
    .getOrCreate()


# Extract a sample JSON string to infer schema
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)


# Create DataFrame representing the stream of input
# lines from connection to logsender 7777
try:
  json_lines = spark.readStream.format("socket") \
      .option("host", "localhost") \
      .option("port", 7777) \
      .load()

   # Parse JSON and extract date components
  parsed_df = json_lines \
      .withColumn("json_data", from_json(col("value"), inferred_schema)) \
      .select("json_data.*") \
      .withColumn("date", to_timestamp("date")) \

  # Get running totals for each day and each sensor
  running_totals = parsed_df \
                            .groupBy(window("date", "1 day"), "sensor",) \
                            .agg(
                                  max("energy").alias("max_energy"),
                                  min("energy").alias("min_energy")
                            ) \
                            .withColumn("sensor_running_total", round(col("max_energy") - col("min_energy"), 4)) \
                            .withColumn("date_s", to_date("window.start")) \

  # Output the results with 5-minute trigger
  query = running_totals \
      .writeStream \
      .outputMode("complete") \
      .trigger(processingTime='5 minutes') \
      .foreachBatch(lambda df, epoch: get_percentages(df).show(50))\
      .start()

  query.awaitTermination(240)
except Exception as err:
  print(err)
  query.stop()

+----+------+----------------+------------------+--------------------+----------------------------+
|date|sensor|percentage_daily|percentage_monthly|sensor_running_total|sensor_monthly_running_total|
+----+------+----------------+------------------+--------------------+----------------------------+
+----+------+----------------+------------------+--------------------+----------------------------+

+----------+------+----------------+------------------+--------------------+----------------------------+
|      date|sensor|percentage_daily|percentage_monthly|sensor_running_total|sensor_monthly_running_total|
+----------+------+----------------+------------------+--------------------+----------------------------+
|2024-10-01|     D|           21.11|             21.11|                 8.8|                         8.8|
|2024-10-01|     H|           11.25|             11.25|                4.69|                        4.69|
|2024-10-01|     A|           11.16|             11.16|              